# Triton Kernel Profiling on NVIDIA GPUs

## Check Nsight Tools Versions

In [5]:
!ncu --version
!nsys --version

NVIDIA (R) Nsight Compute Command Line Profiler
Copyright (c) 2018-2025 NVIDIA Corporation
Version 2025.1.1.0 (build 35528883) (public-release)
NVIDIA Nsight Systems version 2025.1.3.140-251335620677v0


## Check profiling environment

In [6]:
!nsys status -e

Timestamp counter supported: Yes

CPU Profiling Environment Check
Root privilege: enabled
Linux Kernel Paranoid Level = 2
Linux Distribution = CentOS
Linux Kernel Version = 6.15.4-200.fc42.x86_64: OK
Linux perf_event_open syscall available: OK
Sampling trigger event available: OK
Intel(c) Last Branch Record support: Available
CPU Profiling Environment (process-tree): OK
CPU Profiling Environment (system-wide): OK

See the product documentation at https://docs.nvidia.com/nsight-systems for more information,
including information on how to set the Linux Kernel Paranoid Level.


## Python code for the bad MatMul kernel

In [7]:
%%writefile  bad_matmul.py
import torch

import triton
import triton.language as tl


DEVICE = "cuda"


def is_cuda():
    return triton.runtime.driver.active.get_current_target().backend == "cuda"


# Triton Autotuning Configs for NVIDIA
def get_autotune_config():
    return [
        # Bad config
        triton.Config({'BLOCK_SIZE_M': 16, 'BLOCK_SIZE_N': 16, 'BLOCK_SIZE_K': 32, 'GROUP_SIZE_M': 1}, num_stages=3,
                      num_warps=1),
    ]


# MatMul kernel
@triton.autotune(
    configs=get_autotune_config(),
    key=['M', 'N', 'K'],
)
@triton.jit
def matmul_kernel(
        a_ptr, b_ptr, c_ptr,
        M, N, K,
        stride_am, stride_ak,
        stride_bk, stride_bn,
        stride_cm, stride_cn,
        BLOCK_SIZE_M: tl.constexpr, BLOCK_SIZE_N: tl.constexpr, BLOCK_SIZE_K: tl.constexpr,
        GROUP_SIZE_M: tl.constexpr,
):
    pid = tl.program_id(axis=0)
    num_pid_m = tl.cdiv(M, BLOCK_SIZE_M)
    num_pid_n = tl.cdiv(N, BLOCK_SIZE_N)
    num_pid_in_group = GROUP_SIZE_M * num_pid_n
    group_id = pid // num_pid_in_group
    first_pid_m = group_id * GROUP_SIZE_M
    group_size_m = min(num_pid_m - first_pid_m, GROUP_SIZE_M)
    pid_m = first_pid_m + ((pid % num_pid_in_group) % group_size_m)
    pid_n = (pid % num_pid_in_group) // group_size_m

    offs_am = (pid_m * BLOCK_SIZE_M + tl.arange(0, BLOCK_SIZE_M)) % M
    offs_bn = (pid_n * BLOCK_SIZE_N + tl.arange(0, BLOCK_SIZE_N)) % N
    offs_k = tl.arange(0, BLOCK_SIZE_K)
    a_ptrs = a_ptr + (offs_am[:, None] * stride_am + offs_k[None, :] * stride_ak)
    b_ptrs = b_ptr + (offs_k[:, None] * stride_bk + offs_bn[None, :] * stride_bn)

    accumulator = tl.zeros((BLOCK_SIZE_M, BLOCK_SIZE_N), dtype=tl.float32)
    for k in range(0, tl.cdiv(K, BLOCK_SIZE_K)):
        a = tl.load(a_ptrs, mask=offs_k[None, :] < K - k * BLOCK_SIZE_K, other=0.0)
        b = tl.load(b_ptrs, mask=offs_k[:, None] < K - k * BLOCK_SIZE_K, other=0.0)
        accumulator = tl.dot(a, b, accumulator)
        a_ptrs += BLOCK_SIZE_K * stride_ak
        b_ptrs += BLOCK_SIZE_K * stride_bk
    c = accumulator.to(tl.float16)

    offs_cm = pid_m * BLOCK_SIZE_M + tl.arange(0, BLOCK_SIZE_M)
    offs_cn = pid_n * BLOCK_SIZE_N + tl.arange(0, BLOCK_SIZE_N)
    c_ptrs = c_ptr + stride_cm * offs_cm[:, None] + stride_cn * offs_cn[None, :]
    c_mask = (offs_cm[:, None] < M) & (offs_cn[None, :] < N)
    tl.store(c_ptrs, c, mask=c_mask)


# MatMul kernel wrapper function
def matmul(a, b):
    assert a.shape[1] == b.shape[0], "Incompatible dimensions"
    assert a.is_contiguous(), "Matrix A must be contiguous"
    M, K = a.shape
    K, N = b.shape
    c = torch.empty((M, N), device=a.device, dtype=torch.float16)
    grid = lambda META: (triton.cdiv(M, META['BLOCK_SIZE_M']) * triton.cdiv(N, META['BLOCK_SIZE_N']), )
    matmul_kernel[grid](
        a, b, c,
        M, N, K,
        a.stride(0), a.stride(1),
        b.stride(0), b.stride(1),
        c.stride(0), c.stride(1),
    )
    return c


if __name__ == "__main__":
    # Test matrices
    torch.manual_seed(0)
    M = 4096
    N = 4096
    a = torch.randn((M, N), device='cuda', dtype=torch.float16)
    b = torch.randn((N, M), device='cuda', dtype=torch.float16)

    # Run the MatMul kernel
    triton_output = matmul(a, b)
    print(f"triton_output_with_fp16_inputs={triton_output}")

    # Run the Torch MatMul kernel for comparison
    torch_output = torch.matmul(a, b)
    print(f"torch_output_with_fp16_inputs={torch_output}")

    # Verify the Triton kernels results against the Torch kernels
    triton.testing.assert_close(triton_output, torch_output, atol=1e-2, rtol=0)

Writing bad_matmul.py


## Let's profile the bad kernel with Nsight Compute and generate a report file (bad_matmul.ncu-rep)

In [8]:
!ncu --target-processes all --set full --import-source on -f -o bad_matmul python3.12 bad_matmul.py

==PROF== Connected to process 20299 (/usr/bin/python3.12)
==PROF== Profiling "distribution_elementwise_grid..." - 0: 0%....50%....100% - 40 passes
==PROF== Profiling "distribution_elementwise_grid..." - 1: 0%....50%....100% - 40 passes
==PROF== Profiling "matmul_kernel" - 2: 0%....50%....100% - 40 passes
==PROF== Profiling "CatArrayBatchedCopy_contig" - 3: 0%....50%....100% - 40 passes
==PROF== Profiling "CatArrayBatchedCopy_contig" - 4: 0%....50%....100% - 40 passes
==PROF== Profiling "CatArrayBatchedCopy_contig" - 5: 0%....50%....100% - 40 passes
==PROF== Profiling "CatArrayBatchedCopy_contig" - 6: 0%....50%....100% - 40 passes
==PROF== Profiling "CatArrayBatchedCopy_contig" - 7: 0%....50%....100% - 40 passes
==PROF== Profiling "CatArrayBatchedCopy_contig" - 8: 0%....50%....100% - 40 passes
==PROF== Profiling "CatArrayBatchedCopy_contig" - 9: 0%....50%....100% - 40 passes
==PROF== Profiling "vectorized_elementwise_kernel" - 10: 0%....50%....100% - 40 passes
==PROF== Profiling "vector

## Python code for an improved MatMul kernel

In [9]:
%%writefile  improved_matmul.py
import torch

import triton
import triton.language as tl


DEVICE = "cuda"


def is_cuda():
    return triton.runtime.driver.active.get_current_target().backend == "cuda"


# Triton Autotuning Configs for NVIDIA
def get_autotune_config():
    return [
        # Good configs
        triton.Config({'BLOCK_SIZE_M': 128, 'BLOCK_SIZE_N': 256, 'BLOCK_SIZE_K': 64, 'GROUP_SIZE_M': 8}, num_stages=3,
                      num_warps=8),
        triton.Config({'BLOCK_SIZE_M': 64, 'BLOCK_SIZE_N': 256, 'BLOCK_SIZE_K': 32, 'GROUP_SIZE_M': 8}, num_stages=4,
                      num_warps=4),
        triton.Config({'BLOCK_SIZE_M': 128, 'BLOCK_SIZE_N': 128, 'BLOCK_SIZE_K': 32, 'GROUP_SIZE_M': 8}, num_stages=4,
                      num_warps=4),
        triton.Config({'BLOCK_SIZE_M': 128, 'BLOCK_SIZE_N': 64, 'BLOCK_SIZE_K': 32, 'GROUP_SIZE_M': 8}, num_stages=4,
                      num_warps=4),
        triton.Config({'BLOCK_SIZE_M': 64, 'BLOCK_SIZE_N': 128, 'BLOCK_SIZE_K': 32, 'GROUP_SIZE_M': 8}, num_stages=4,
                      num_warps=4),
        triton.Config({'BLOCK_SIZE_M': 128, 'BLOCK_SIZE_N': 32, 'BLOCK_SIZE_K': 32, 'GROUP_SIZE_M': 8}, num_stages=4,
                      num_warps=4),
        triton.Config({'BLOCK_SIZE_M': 64, 'BLOCK_SIZE_N': 32, 'BLOCK_SIZE_K': 32, 'GROUP_SIZE_M': 8}, num_stages=5,
                      num_warps=2),
        triton.Config({'BLOCK_SIZE_M': 32, 'BLOCK_SIZE_N': 64, 'BLOCK_SIZE_K': 32, 'GROUP_SIZE_M': 8}, num_stages=5,
                      num_warps=2),
    ]


# MatMul kernel
@triton.autotune(
    configs=get_autotune_config(),
    key=['M', 'N', 'K'],
)
@triton.jit
def matmul_kernel(
        a_ptr, b_ptr, c_ptr,
        M, N, K,
        stride_am, stride_ak,
        stride_bk, stride_bn,
        stride_cm, stride_cn,
        BLOCK_SIZE_M: tl.constexpr, BLOCK_SIZE_N: tl.constexpr, BLOCK_SIZE_K: tl.constexpr,
        GROUP_SIZE_M: tl.constexpr,
):
    pid = tl.program_id(axis=0)
    num_pid_m = tl.cdiv(M, BLOCK_SIZE_M)
    num_pid_n = tl.cdiv(N, BLOCK_SIZE_N)
    num_pid_in_group = GROUP_SIZE_M * num_pid_n
    group_id = pid // num_pid_in_group
    first_pid_m = group_id * GROUP_SIZE_M
    group_size_m = min(num_pid_m - first_pid_m, GROUP_SIZE_M)
    pid_m = first_pid_m + ((pid % num_pid_in_group) % group_size_m)
    pid_n = (pid % num_pid_in_group) // group_size_m

    offs_am = (pid_m * BLOCK_SIZE_M + tl.arange(0, BLOCK_SIZE_M)) % M
    offs_bn = (pid_n * BLOCK_SIZE_N + tl.arange(0, BLOCK_SIZE_N)) % N
    offs_k = tl.arange(0, BLOCK_SIZE_K)
    a_ptrs = a_ptr + (offs_am[:, None] * stride_am + offs_k[None, :] * stride_ak)
    b_ptrs = b_ptr + (offs_k[:, None] * stride_bk + offs_bn[None, :] * stride_bn)

    accumulator = tl.zeros((BLOCK_SIZE_M, BLOCK_SIZE_N), dtype=tl.float32)
    for k in range(0, tl.cdiv(K, BLOCK_SIZE_K)):
        a = tl.load(a_ptrs, mask=offs_k[None, :] < K - k * BLOCK_SIZE_K, other=0.0)
        b = tl.load(b_ptrs, mask=offs_k[:, None] < K - k * BLOCK_SIZE_K, other=0.0)
        accumulator = tl.dot(a, b, accumulator)
        a_ptrs += BLOCK_SIZE_K * stride_ak
        b_ptrs += BLOCK_SIZE_K * stride_bk
    c = accumulator.to(tl.float16)

    offs_cm = pid_m * BLOCK_SIZE_M + tl.arange(0, BLOCK_SIZE_M)
    offs_cn = pid_n * BLOCK_SIZE_N + tl.arange(0, BLOCK_SIZE_N)
    c_ptrs = c_ptr + stride_cm * offs_cm[:, None] + stride_cn * offs_cn[None, :]
    c_mask = (offs_cm[:, None] < M) & (offs_cn[None, :] < N)
    tl.store(c_ptrs, c, mask=c_mask)


# MatMul kernel wrapper function
def matmul(a, b):
    assert a.shape[1] == b.shape[0], "Incompatible dimensions"
    assert a.is_contiguous(), "Matrix A must be contiguous"
    M, K = a.shape
    K, N = b.shape
    c = torch.empty((M, N), device=a.device, dtype=torch.float16)
    grid = lambda META: (triton.cdiv(M, META['BLOCK_SIZE_M']) * triton.cdiv(N, META['BLOCK_SIZE_N']), )
    matmul_kernel[grid](
        a, b, c,
        M, N, K,
        a.stride(0), a.stride(1),
        b.stride(0), b.stride(1),
        c.stride(0), c.stride(1),
    )
    return c


if __name__ == "__main__":
    # Test matrices
    torch.manual_seed(0)
    M = 4096
    N = 4096
    a = torch.randn((M, N), device='cuda', dtype=torch.float16)
    b = torch.randn((N, M), device='cuda', dtype=torch.float16)

    # Run the MatMul kernel
    triton_output = matmul(a, b)
    print(f"triton_output_with_fp16_inputs={triton_output}")

    # Run the Torch MatMul kernel for comparison
    torch_output = torch.matmul(a, b)
    print(f"torch_output_with_fp16_inputs={torch_output}")

    # Verify the Triton kernels results against the Torch kernels
    triton.testing.assert_close(triton_output, torch_output, atol=1e-2, rtol=0)

Writing improved_matmul.py


## Let's profile the improved kernel with Nsight Compute and generate a report file (improved_matmul.ncu-rep)

In [10]:
!ncu --target-processes all --set full --import-source on -f -o improved_matmul python3.12 improved_matmul.py

==PROF== Connected to process 22840 (/usr/bin/python3.12)
==PROF== Profiling "distribution_elementwise_grid..." - 0: 0%....50%....100% - 40 passes
==PROF== Profiling "distribution_elementwise_grid..." - 1: 0%....50%....100% - 40 passes
==PROF== Profiling "matmul_kernel" - 2: 0%....50%....100% - 40 passes
==PROF== Profiling "vectorized_elementwise_kernel" - 3: 0%....50%....100% - 40 passes
==PROF== Profiling "matmul_kernel" - 4: 0%....50%....100% - 40 passes
==PROF== Profiling "vectorized_elementwise_kernel" - 5: 0%....50%....100% - 40 passes
==PROF== Profiling "matmul_kernel" - 6: 0%....50%....100% - 40 passes
==PROF== Profiling "vectorized_elementwise_kernel" - 7: 0%....50%....100% - 40 passes
==PROF== Profiling "matmul_kernel" - 8: 0%....50%....100% - 40 passes
==PROF== Profiling "vectorized_elementwise_kernel" - 9: 0%....50%....100% - 40 passes
==PROF== Profiling "matmul_kernel" - 10: 0%....50%....100% - 40 passes
==PROF== Profiling "vectorized_elementwise_kernel" - 11: 0%....50%...